## 0. Import

In [3]:
import numpy as np
import pandas as pd

In [4]:
import re

In [5]:
data_ps = pd.read_csv('C:/Users/stefa/Google Drive/DATA SCIENCE/Foundation of Computer Science/Project/google_play_store.csv')
data_rev = pd.read_csv('C:/Users/stefa/Google Drive/DATA SCIENCE/Foundation of Computer Science/Project/google_play_store_reviews.csv')

In [6]:
data_ps = data_ps.drop(10472)

In [7]:
data_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
data_rev.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


## 1. Convert the app sizes to a number

In [9]:
size_to_int_re = re.compile('^(?P<number>\d+\.?\d*)(?P<unit>\w*)$')

def to_mult(unit):
    if (unit == 'G'):
        return 1000000000
    if (unit == 'M'):
        return 1000000
    if (unit == 'k'):
        return 1000
    return 1

def size_to_num(size):
    num = size_to_int_re.match(size)
    if num:
        unit = num.group('unit')
        mult = to_mult(unit)
        size = float(num.group('number')) * mult
    return size

In [10]:
data_ps['Size'] = data_ps['Size'].apply(size_to_num)
data_ps['Size'].head()

0    1.9e+07
1    1.4e+07
2    8.7e+06
3    2.5e+07
4    2.8e+06
Name: Size, dtype: object

## 2. Convert the number of installs to a number

In [11]:
data_ps['Installs'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
data_ps.Installs = data_ps.Installs.astype(float)

In [12]:
data_ps['Installs'].head()

0       10000.0
1      500000.0
2     5000000.0
3    50000000.0
4      100000.0
Name: Installs, dtype: float64

## 3. Transform “Varies with device” into a missing value

In [13]:
def transform_varies(value):
    if value == 'Varies with device':
        value = None
    return value

In [14]:
colonne = list(data_ps.columns)
for colonna in colonne:
    data_ps[colonna] = data_ps[colonna].apply(transform_varies)

In [15]:
print(data_ps['Size'].isnull().sum())
print(data_ps['Current Ver'].isnull().sum())
print(data_ps['Android Ver'].isnull().sum())

1695
1467
1364


## 4. Convert Current Ver and Android Ver into a dotted number

In [16]:
data_ps.groupby('Android Ver').size()

Android Ver
1.0 and up          2
1.5 and up         20
1.6 and up        116
2.0 and up         32
2.0.1 and up        7
2.1 and up        134
2.2 - 7.1.1         1
2.2 and up        244
2.3 and up        652
2.3.3 and up      281
3.0 and up        241
3.1 and up         10
3.2 and up         36
4.0 and up       1375
4.0.3 - 7.1.1       2
4.0.3 and up     1501
4.1 - 7.1.1         1
4.1 and up       2451
4.2 and up        394
4.3 and up        243
4.4 and up        980
4.4W and up        12
5.0 - 6.0           1
5.0 - 7.1.1         1
5.0 - 8.0           2
5.0 and up        601
5.1 and up         24
6.0 and up         60
7.0 - 7.1.1         1
7.0 and up         42
7.1 and up          3
8.0 and up          6
dtype: int64

In [17]:
# se c'è trattino prendi solo secondo, altrimenti prendi solo numeri

In [18]:
data_ps['Android Ver'] = data_ps['Android Ver'].str.split('-').str[-1]

In [19]:
data_ps['Android Ver'].replace(regex=True,inplace=True,to_replace=r'[^\d\.]',value=r'')

In [20]:
data_ps.groupby('Android Ver').size()

Android Ver
1.0         2
1.5        20
1.6       116
2.0        32
2.0.1       7
2.1       134
2.2       244
2.3       652
2.3.3     281
3.0       241
3.1        10
3.2        36
4.0      1375
4.0.3    1501
4.1      2451
4.2       394
4.3       243
4.4       992
5.0       601
5.1        24
6.0        61
7.0        42
7.1         3
7.1.1       6
8.0         8
dtype: int64

In [21]:
data_ps.groupby('Current Ver').size()

Current Ver
0.0.0.2                      1
0.0.1                       15
0.0.10                       1
0.0.2                        4
0.0.3                        2
0.0.4                        3
0.0.42                       1
0.0.44                       1
0.0.5                        3
0.0.52                       1
0.0.53                       1
0.0.6                        1
0.0.7                        3
0.0.73                       1
0.0.80                       1
0.0.9                        5
0.1                         13
0.1.0                        2
0.1.1                        3
0.1.10.0                     1
0.1.100944346                2
0.1.11                       1
0.1.187945513                1
0.1.2                        1
0.1.219                      1
0.1.22                       1
0.1.27                       1
0.1.5                        1
0.1.6                        1
0.1.7                        1
                            ..
v150                       

In [22]:
# togli tutto tranne numeri e punti

data_ps['Current Ver'].replace(regex=True,inplace=True,to_replace=r'[^\d\.]',value=r'')

In [23]:
data_ps.groupby('Current Ver').size()  # togli tutto tranne numeri e punti

Current Ver
                    16
.                    1
.1.0                 1
.2                   1
.2014.002            1
.2918664             1
.6                   1
0.0.0.2              1
0.0.1               15
0.0.10               1
0.0.2                4
0.0.3                2
0.0.4                3
0.0.42               1
0.0.44               1
0.0.5                3
0.0.52               1
0.0.53               1
0.0.6                1
0.0.7                3
0.0.73               1
0.0.80               1
0.0.9                5
0.1                 13
0.1.0                2
0.1.1                3
0.1.10.0             1
0.1.100944346        2
0.1.11               1
0.1.187945513        1
                    ..
9.2.4                1
9.3.1                3
9.3.3                1
9.3.52               1
9.3.5776             1
9.3.6                1
9.4.0                1
9.4.1.3              1
9.4.2                1
9.4.7.6              1
9.40.3               2
9.5.0                1

In [24]:
# assegna None a valori anomali (mancanti o che iniziano con un punto)

data_ps['Current Ver'][data_ps['Current Ver'].str.startswith(('.'),na=False)] = None
data_ps['Current Ver'][data_ps['Current Ver'] == ''] = None

C:\Users\stefa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\stefa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [25]:
data_ps.groupby('Current Ver').size()

Current Ver
0.0.0.2              1
0.0.1               15
0.0.10               1
0.0.2                4
0.0.3                2
0.0.4                3
0.0.42               1
0.0.44               1
0.0.5                3
0.0.52               1
0.0.53               1
0.0.6                1
0.0.7                3
0.0.73               1
0.0.80               1
0.0.9                5
0.1                 13
0.1.0                2
0.1.1                3
0.1.10.0             1
0.1.100944346        2
0.1.11               1
0.1.187945513        1
0.1.2                1
0.1.219              1
0.1.22               1
0.1.27               1
0.1.5                1
0.1.6                1
0.1.7                1
                    ..
9.2.4                1
9.3.1                3
9.3.3                1
9.3.52               1
9.3.5776             1
9.3.6                1
9.4.0                1
9.4.1.3              1
9.4.2                1
9.4.7.6              1
9.40.3               2
9.5.0                1

## 5. Remove the duplicates

In [26]:
data_ps['Reviews'] = data_ps['Reviews'].astype(int)

app_dupl = data_ps.sort_values(by=['App','Reviews','Category'],ascending=False)

app_dupl = app_dupl[app_dupl.duplicated(subset='App', keep='first')]

In [225]:
data_ps = data_ps.drop(app_dupl.index)

## 6. For each category, compute the number of apps

In [226]:
data_ps.groupby('Category')['Category'].count()

Category
ART_AND_DESIGN           61
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     222
BUSINESS                420
COMICS                   56
COMMUNICATION           315
DATING                  170
EDUCATION               105
ENTERTAINMENT            86
EVENTS                   64
FAMILY                 1877
FINANCE                 345
FOOD_AND_DRINK          112
GAME                    946
HEALTH_AND_FITNESS      288
HOUSE_AND_HOME           73
LIBRARIES_AND_DEMO       84
LIFESTYLE               369
MAPS_AND_NAVIGATION     131
MEDICAL                 395
NEWS_AND_MAGAZINES      254
PARENTING                60
PERSONALIZATION         376
PHOTOGRAPHY             281
PRODUCTIVITY            374
SHOPPING                202
SOCIAL                  239
SPORTS                  325
TOOLS                   829
TRAVEL_AND_LOCAL        219
VIDEO_PLAYERS           164
WEATHER                  79
Name: Category, dtype: int64

## 7. For each category, compute the average rating

In [227]:
data_ps.groupby('Category')['Rating'].mean()

Category
ART_AND_DESIGN         4.359322
AUTO_AND_VEHICLES      4.190411
BEAUTY                 4.278571
BOOKS_AND_REFERENCE    4.344970
BUSINESS               4.098479
COMICS                 4.181481
COMMUNICATION          4.121484
DATING                 3.980451
EDUCATION              4.349038
ENTERTAINMENT          4.129070
EVENTS                 4.435556
FAMILY                 4.184019
FINANCE                4.115563
FOOD_AND_DRINK         4.171277
GAME                   4.244605
HEALTH_AND_FITNESS     4.243033
HOUSE_AND_HOME         4.140984
LIBRARIES_AND_DEMO     4.178125
LIFESTYLE              4.093355
MAPS_AND_NAVIGATION    4.036441
MEDICAL                4.165862
NEWS_AND_MAGAZINES     4.121569
PARENTING              4.300000
PERSONALIZATION        4.332215
PHOTOGRAPHY            4.155894
PRODUCTIVITY           4.183389
SHOPPING               4.230556
SOCIAL                 4.247291
SPORTS                 4.216154
TOOLS                  4.040278
TRAVEL_AND_LOCAL       4.069519

## 8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [28]:
new = data_ps['Genres'].str.split(';',1,expand=True)

In [229]:
data_ps['Genre_1'] = new[0]
data_ps['Genre_2'] = new[1]

In [230]:
gen1 = data_ps[['App','Genre_1']]
gen2 = data_ps[['App','Genre_2']].dropna()

In [231]:
gen1 = gen1.rename(columns = {'Genre_1':'Gen'})
gen2 = gen2.rename(columns = {'Genre_2':'Gen'})

In [232]:
bridging = pd.concat([gen1,gen2]).drop_duplicates()

In [233]:
bridging[bridging['App']=='Pixel Draw - Number Art Coloring Book'] #check

,App,Gen
4,Pixel Draw - Number Art Coloring Book,Art & Design
4,Pixel Draw - Number Art Coloring Book,Creativity


In [234]:
genres_df = pd.DataFrame(bridging['Gen'].drop_duplicates())
genres_df = genres_df.reset_index(drop=True)
genres_df

,Gen
0,Art & Design
1,Auto & Vehicles
2,Beauty
3,Books & Reference
4,Business
5,Comics
6,Communication
7,Dating
8,Education
9,Entertainment


## 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [235]:
df_temp = pd.DataFrame(columns = genres_df['Gen'])

data_ps = data_ps.join(df_temp)

In [236]:
for i, row in data_ps.iterrows():
    for k in genres_df['Gen']:
        data_ps.at[i,k] = False
    j = row['Genres'].split(';')
    for genre in j:
        data_ps.at[i,genre] = True

In [246]:
pd.options.display.max_columns = 999
data_ps[data_ps['App']=='Pixel Draw - Number Art Coloring Book'] #check

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Genre_1,Genre_2,Art & Design,Auto & Vehicles,Beauty,Books & Reference,Business,Comics,Communication,Dating,Education,Entertainment,Events,Finance,Food & Drink,Health & Fitness,House & Home,Libraries & Demo,Lifestyle,Card,Arcade,Puzzle,Racing,Sports,Casual,Simulation,Adventure,Trivia,Action,Word,Role Playing,Strategy,Board,Music,Educational,Music & Audio,Medical,Social,Shopping,Photography,Travel & Local,Tools,Personalization,Productivity,Parenting,Weather,Video Players & Editors,News & Magazines,Maps & Navigation,Casino,Creativity,Pretend Play,Brain Games,Music & Video,Action & Adventure,Income
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000.0,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4,Art & Design,Creativity,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.0


## 10. For each genre, compute the average rating. What is the genre with highest average?

In [237]:
df_ten = pd.merge(bridging,data_ps,on='App')
df_ten

,App,Gen,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,...,Weather,Video Players & Editors,News & Magazines,Maps & Navigation,Casino,Creativity,Pretend Play,Brain Games,Music & Video,Action & Adventure
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design,ART_AND_DESIGN,4.1,159,19000000.0,10000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design,ART_AND_DESIGN,4.7,87510,8700000.0,5000000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
2,Sketch - Draw & Paint,Art & Design,ART_AND_DESIGN,4.5,215644,25000000.0,50000000.0,Free,0,Teen,...,False,False,False,False,False,False,False,False,False,False
3,Pixel Draw - Number Art Coloring Book,Art & Design,ART_AND_DESIGN,4.3,967,2800000.0,100000.0,Free,0,Everyone,...,False,False,False,False,False,True,False,False,False,False
4,Pixel Draw - Number Art Coloring Book,Creativity,ART_AND_DESIGN,4.3,967,2800000.0,100000.0,Free,0,Everyone,...,False,False,False,False,False,True,False,False,False,False
5,Paper flowers instructions,Art & Design,ART_AND_DESIGN,4.4,167,5600000.0,50000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
6,Smoke Effect Photo Maker - Smoke Editor,Art & Design,ART_AND_DESIGN,3.8,178,19000000.0,50000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
7,Infinite Painter,Art & Design,ART_AND_DESIGN,4.1,36815,29000000.0,1000000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
8,Garden Coloring Book,Art & Design,ART_AND_DESIGN,4.4,13791,33000000.0,1000000.0,Free,0,Everyone,...,False,False,False,False,False,False,False,False,False,False
9,Kids Paint Free - Drawing Fun,Art & Design,ART_AND_DESIGN,4.7,121,3100000.0,10000.0,Free,0,Everyone,...,False,False,False,False,False,True,False,False,False,False


In [238]:
df_ten.groupby('Gen')['Rating'].mean()

Gen
Action                     4.247697
Action & Adventure         4.288542
Adventure                  4.191026
Arcade                     4.277838
Art & Design               4.350000
Auto & Vehicles            4.190411
Beauty                     4.278571
Board                      4.291228
Books & Reference          4.343275
Brain Games                4.358065
Business                   4.098479
Card                       4.082609
Casino                     4.286486
Casual                     4.141014
Comics                     4.181481
Communication              4.121790
Creativity                 4.306250
Dating                     3.980451
Education                  4.290018
Educational                4.100000
Entertainment              4.093028
Events                     4.435556
Finance                    4.115563
Food & Drink               4.171277
Health & Fitness           4.243496
House & Home               4.140984
Libraries & Demo           4.178125
Lifestyle               

In [239]:
df_ten.groupby('Gen')['Rating'].mean().idxmax()

'Events'

## 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

In [240]:
data_ps['Price'].replace(regex=True,inplace=True,to_replace=r'[^\d\.]',value=r'')
data_ps.Price = data_ps.Price.astype(float)

In [241]:
data_ps['Income'] = data_ps['Installs'] * data_ps['Price']

In [242]:
data_ps[data_ps['Income'] != 0].head(10) #check

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Video Players & Editors,News & Magazines,Maps & Navigation,Casino,Creativity,Pretend Play,Brain Games,Music & Video,Action & Adventure,Income
234,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6800000.0,100000.0,Paid,4.99,Everyone,Business,...,False,False,False,False,False,False,False,False,False,499000.0
235,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39000000.0,100000.0,Paid,4.99,Everyone,Business,...,False,False,False,False,False,False,False,False,False,499000.0
427,Puffin Browser Pro,COMMUNICATION,4.0,18247,NaN,100000.0,Paid,3.99,Everyone,Communication,...,False,False,False,False,False,False,False,False,False,399000.0
478,Truth or Dare Pro,DATING,NaN,0,20000000.0,50.0,Paid,1.49,Teen,Dating,...,False,False,False,False,False,False,False,False,False,74.5
479,"Private Dating, Hide App- Blue for PrivacyHider",DATING,NaN,0,18000.0,100.0,Paid,2.99,Everyone,Dating,...,False,False,False,False,False,False,False,False,False,299.0
480,Ad Blocker for SayHi,DATING,NaN,4,1200000.0,100.0,Paid,3.99,Teen,Dating,...,False,False,False,False,False,False,False,False,False,399.0
481,AMBW Dating App: Asian Men Black Women Interra...,DATING,3.5,2,17000000.0,100.0,Paid,7.99,Mature 17+,Dating,...,False,False,False,False,False,False,False,False,False,799.0
571,"Moco+ - Chat, Meet People",DATING,4.2,1546,NaN,10000.0,Paid,3.99,Mature 17+,Dating,...,False,False,False,False,False,False,False,False,False,39900.0
851,Sago Mini Hat Maker,EDUCATION,4.9,11,63000000.0,1000.0,Paid,3.99,Everyone,Education;Pretend Play,...,False,False,False,False,False,True,False,False,False,3990.0
854,Toca Life: Hospital,EDUCATION,4.7,3528,24000000.0,100000.0,Paid,3.99,Everyone,Education;Pretend Play,...,False,False,False,False,False,True,False,False,False,399000.0


## 12. For each app, compute its minimum and maximum Sentiment_polarity.

In [243]:
sentiment_df = pd.DataFrame({ 'Sentiment_max' : data_rev.groupby('App')['Sentiment_Polarity'].max(), 
                             'Sentiment_min' : data_rev.groupby('App')['Sentiment_Polarity'].min()}).reset_index()
sentiment_df.head(10)

,App,Sentiment_max,Sentiment_min
0,10 Best Foods for You,1.000000,-0.800000
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.112500
2,11st,1.000000,-1.000000
3,1800 Contacts - Lens Store,0.838542,-0.300000
4,1LINE – One Line with One Touch,1.000000,-0.825000
5,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000,-0.800000
6,21-Day Meditation Experience,0.587500,-0.265625
7,"2Date Dating App, Love and matching",1.000000,-0.645833
8,2GIS: directory & navigator,1.000000,-0.375000
9,2RedBeans,1.000000,-0.800000
